# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [3]:
finetuning = 1
load_and_run = not finetuning

# Load Data

In [4]:
import pandas as pd

data = pd.read_csv('/kaggle/input/gb-all-necessary/train_test_toxicity_data.csv')
data

,male,female,_original_label,test
0,চোদনার পুত বদনার নাতি।। সালা জাত ফকিন্নি।। মাদ...,চোদনার মেয়া বদনার নাতনি । । সালী জাত ফকির । ...,1,0
1,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,1,0
2,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",1,0
3,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",1,0
4,যদি বদরুল শিবিরের নেতা হয় তাহলে ফাসি দেওয়া হক ...,যদি বদরুল শিব িরের নেত্রী হয় তাহলে ফাসি দেওয়া...,1,0
...,...,...,...,...
7047,নিশো ভাই তুমি সেরাদারুন নাটক,অমৃতা আফা তুমি সেরাদারুন নাটক,0,0
7048,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,0,1
7049,নিশো ভাই তুমি সেরাদারুন নাটক,দুর্গা ভাবী তুমি সেরাদারুন নাটক,0,1
7050,সত্যি বলতে কি এই রকম পাগল টাইপের ভালোবাসা আমার...,সত্যি বলতে কি এই রকম পাগলী টাইপের ভালোবাসা আমা...,0,0


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [5]:
temp = data[data['test']!= 1]
test = data[data['test']== 1]
validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))
train = temp.drop(validation.index)

<ipython-input-5-d28a0f0bc0b1>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))


In [6]:
train

,male,female,_original_label,test
0,চোদনার পুত বদনার নাতি।। সালা জাত ফকিন্নি।। মাদ...,চোদনার মেয়া বদনার নাতনি । । সালী জাত ফকির । ...,1,0
1,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,1,0
2,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",1,0
3,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",1,0
5,ছোটকা কী পারবে শ্রেষ্ঠা কে তুলে নিয়ে গিয়ে মাঝ ...,ছোটকী কী পারবে শ্রেষ্ঠা কে তুলে নিয়ে গিয়ে মাঝ ...,1,0
...,...,...,...,...
7045,ভালো লাগছে নাটকটা কিন্তু ১০ বছর বয়সে ছেলে কলেজ...,ভালো লাগছে নাটকটা কিন্তু ১০ বছর বয়সে মেয়ে কলেজ...,0,0
7046,অসম্ভব সুন্দর এত ভালো লেগেছে নাটকটা অনেকদিন পর...,অসম্ভব সুন্দর এত ভালো লেগেছে নাটকটা অনেকদিন পর...,0,0
7047,নিশো ভাই তুমি সেরাদারুন নাটক,অমৃতা আফা তুমি সেরাদারুন নাটক,0,0
7050,সত্যি বলতে কি এই রকম পাগল টাইপের ভালোবাসা আমার...,সত্যি বলতে কি এই রকম পাগলী টাইপের ভালোবাসা আমা...,0,0


In [7]:
validation

,male,female,_original_label,test
3679,এখনো এমন অনেক প্রকৃত ভালবাসা রয়েছে কিন্তু কিছু...,এখনো এমন অনেক প্রকৃত ভালবাসা রয়েছে কিন্তু কিছু...,0,0
4718,ভাই অাপনাকে অনেক ধ্যনবাদ অন্য রকম ভাবে দেখলাম ...,আফা অাপনাকে অনেক ধ্যনবাদ অন্য রকম ভাবে দেখলাম ...,0,0
5112,নিশো ভাই আপনে পারেন ভাই পারেন নাটক টা সেই রোমা...,নাজনিন বোন আপনে পারেন বোন পারেন নাটক টা সেই রো...,0,0
6069,বাবা মায়ের ভালবাসার কাছে নিজের ভালবাসা এভাবেই ...,মা বাবার ভালবাসার কাছে নিজের ভালবাসা এভাবেই হা...,0,0
6420,ড্রাইভার এ জার সাথে কথা বলছে সে হলো আমাদের প্র...,ড্রাইভার এ জার সাথে কথা বলছে সে হলো আমাদের প্র...,0,0
...,...,...,...,...
1135,এসব পুরুষের দরকার বাংলাদেশের সবচেয়ে ভালো রড ব...,এসব মহিলার দরকার বাংলাদেশের সবচেয়ে ভালো রড ...,1,0
1465,পুলিশ তো সব ক্ষেত্রে সাহায্য করতে আসে না আর মহ...,মহিলা পুলিশ তো সব ক্ষেত্রে সাহায্য করতে আসে না...,1,0
2790,ভাই আপনি কিছুক্ষণ এর জন্য প্যান্ট খুলে মৌমাছির...,আফা আপনি কিছুক্ষণ এর জন্য প্যান্ট খুলে মৌমাছির...,1,0
2918,ভাজ্ঞিস তোদের ব্রিটিশরা সভ্য করেছিলো আর এখন আস...,ভাজ্ঞিস তোদের ব্রিটিশরা সভ্যা করেছিলো আর এখন আ...,1,0


In [8]:
test

,male,female,_original_label,test
6,"কি দরকার ছিল এই ভিডিও বানানোর,পেটের দায়ে শরীর ...","কি দরকার ছিল এই ভিডিও বানানোর , পেটের দায়ে শরী...",1,1
7,চুদির ভাই বাংলায় লিখতে পারো নাই,চুদির আফা বাংলায় লিখতে পারো নাই,1,1
10,"যখন কেউ বলে অস্ত্রের চাইতে কলম বড়, তখন অস্ত্রধ...","যখন কেউ বলে অস্ত্রের চাইতে কলম বড় , তখন অস্ত্...",1,1
14,পরে দেখা গেলো বললেন বাঙ্গির ব্যাবসা । আপনারা ম...,পরে দেখা গেলো বললেন বাঙ্গির ব্যাবসা । আপনারা...,1,1
16,নাস্তিকদের বর্তমানের অন্যতম পুঁজি কোরানের একটি...,নাস্তিকদের বর্তমানের অন্যতম পুঁজি কোরানের একটি...,1,1
...,...,...,...,...
7032,নাটকটা সত্যিই অনেক অসাধারণ হয়েছে কিন্তু এখানে...,নাটকটা সত্যিই অনেক অসাধারণ হয়েছে কিন্তু এখানে...,0,1
7033,তার লাহোর প্রস্তাবেই ছিল স্বাধীন বাংলাদেশের বী...,তার লাহোর প্রস্তাবেই ছিল স্বাধীন বাংলাদেশের বী...,0,1
7037,নায়ক হিসেবে তাহসান ভাই থাকলে ভালো হতে,নায়িকা হিসেবে বিনিতা আফা থাকলে ভালো হতে,0,1
7048,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,0,1


# Minimizing KL Divergence

## Load Pretrained Model

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model
model_path = "FredZhang7/one-for-all-toxicity-v3"
tokenizer, b_model = load_transformer_based_model(model_path)
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output

Total Pretrained tokens:  119547


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Total number of tokens after adding the new ones:  119549


## Creating Custom Data Loader

In [10]:
class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,2]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th column male text
        text2 = self.train_csv.iloc[index,1] #1st column female text

        inputs = self.tokenizer.batch_encode_plus(
            [text1, text2] ,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        masks = inputs["attention_mask"]

        return torch.tensor(ids[0], dtype=torch.long), torch.tensor(token_type_ids[0], dtype=torch.long), torch.tensor(masks[0], dtype=torch.long), torch.tensor(ids[1], dtype=torch.long), torch.tensor(token_type_ids[1], dtype=torch.long), torch.tensor(masks[1], dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 2], dtype=torch.long)

BATCH_SIZE = 4

dataset_train= BertDataset(train, tokenizer, max_length=512 )
dataset_val= BertDataset( validation, tokenizer, max_length=512)
dataset_test= BertDataset(test, tokenizer, max_length=512)
train_loader=DataLoader(dataset=dataset_train,batch_size=BATCH_SIZE, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=BATCH_SIZE, shuffle=False)
test_loader=DataLoader(dataset=dataset_test,batch_size=BATCH_SIZE, shuffle=False)

## Creating Custom Model

In [11]:
class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        # print(self.bert_model.bert.embeddings.word_embeddings.weight)
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 2)

    def forward(self,ids1, mask1, token_type_ids1, ids2, mask2, token_type_ids2, mode):
        if mode == 'training':
          ids = torch.vstack((ids1, ids2))
          mask = torch.vstack((mask1, mask2))
          token_type_ids = torch.vstack((token_type_ids1, token_type_ids2))
          o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
          o2= o2.max(dim=1).values
          male_embedding, female_embedding = torch.vsplit(o2, 2)
          # print(original_embedding.shape, ner_embedding.shape)
          o2_male = self.dropout(male_embedding)
          out_male= self.out(o2_male)

          o2_female = self.dropout(female_embedding)
          out_female= self.out(o2_female)


          # print(F.softmax(out_male, dim=-1), '\n', F.softmax(out_female, dim=-1))

          return out_male, F.softmax(out_male, dim=-1), F.softmax(out_female, dim=-1)


        if mode == 'inference':
          o2= self.bert_model(ids1, mask1, token_type_ids1).hidden_states[0]
          oe= o2.max(dim=1).values
          o2 = self.dropout(oe)
          out= self.out(o2)
          return out, oe

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
masked_model = nn.DataParallel(BERT(b_model)).to(device)

In [12]:
#debug
i1, t1,m1,i2,t2,m2,l = next(iter(train_loader))
i1 = i1.to(device)
t1 = t1.to(device)
m1 = m1.to(device)
i2 = i2.to(device)
t2 = t2.to(device)
m2 = m2.to(device)
l = l.to(device)
print('Input Shape: ', i1.shape)

logits, male_softmax, female_softmax = masked_model(i1, t1,m1,i2,t2,m2, 'training')
print(logits.shape, male_softmax.shape, male_softmax.shape)
logits, embedding = masked_model(i1, t1,m1,None,None,None, 'inference')
print(logits.shape, embedding.shape)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Input Shape:  torch.Size([4, 512])
torch.Size([4, 2]) torch.Size([4, 2]) torch.Size([4, 2])
torch.Size([4, 2]) torch.Size([4, 768])


In [ ]:
logits, logits.max(1)[1].view(-1,1)

In [ ]:
logits.max(1)[1].view(-1,1).eq(l.view(-1,1)).sum().item()

## Creating Utility Class for Tracking Training Stat

In [13]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_ce_loss = 0
        self.epoch_kl_loss = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_ce_loss = 0.0
        self.epoch_valid_kl_loss = 0.0
        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_ce_loss = 0
        self.epoch_kl_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_ce_loss = 0
        self.epoch_valid_kl_loss = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        ce_loss = self.epoch_ce_loss/len(self.loader.dataset)
        kl_loss = self.epoch_kl_loss/len(self.loader.dataset)
        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_ce_loss = self.epoch_valid_ce_loss/len(self.validation_loader.dataset)
        val_kl_loss = self.epoch_valid_kl_loss/len(self.validation_loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('CE Loss', ce_loss, self.epoch_count)
        self.tb.add_scalar('KL Loss', kl_loss, self.epoch_count)
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation CE Loss', val_ce_loss, self.epoch_count)
        self.tb.add_scalar('Validation KL Loss', val_kl_loss, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)
        print('-------------')
        for name, param in self.network.named_parameters():
            # print(name, param, self.epoch_count)
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["ce_loss"] = ce_loss
        results["kl_loss"] = kl_loss
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation ce loss"] = val_ce_loss
        results["validation kl loss"] = val_kl_loss
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration (minutes)"] = epoch_duration/60
        results["run duration (minutes)"] = run_duration/60
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, ce_loss, kl_loss, loss):
        self.epoch_ce_loss += ce_loss.item()#*self.loader.batch_size
        self.epoch_kl_loss += kl_loss.item()#*self.loader.batch_size
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, ce_loss, kl_loss, loss):
        self.epoch_valid_ce_loss += ce_loss.item()#*self.loader.batch_size
        self.epoch_valid_kl_loss += kl_loss.item()#*self.loader.batch_size
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return preds.max(1)[1].view(-1,1).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')

## Creating Training Loop

In [14]:
SAVE_MODEL = True
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        # network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.CrossEntropyLoss()
        loss_fn_2 = nn.KLDivLoss(reduction="batchmean")

        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = 'Toxicity_KLDM.pth')
        m.begin_run(run, network, train_loader, validation_loader)


        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()


            for batch in tqdm(train_loader):
                input_ids1, token_type_ids1, attention_masks1, input_ids2, token_type_ids2, attention_masks2, labels = batch
                input_ids1 = input_ids1.to(device)
                token_type_ids1 = token_type_ids1.to(device)
                attention_masks1 = attention_masks1.to(device)

                input_ids2 = input_ids2.to(device)
                token_type_ids2 = token_type_ids2.to(device)
                attention_masks2 = attention_masks2.to(device)

                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                copy_labels = labels

                preds, male_softmax, female_softmax = network(input_ids1, attention_masks1, token_type_ids1, input_ids2, attention_masks2, token_type_ids2, 'training')
                labels = labels.view(-1, 1)



                loss1 = loss_fn(preds, copy_labels)
                loss2 = loss_fn_2(male_softmax.log(), female_softmax) + loss_fn_2(female_softmax.log(), male_softmax)
                loss = loss1 + loss2
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)

                optimizer.step()

                scheduler.step()


                m.track_loss(loss1, loss2, loss)
                m.track_num_correct(preds, labels)


            network.eval()
            # print(network.bert_model.bert.embeddings.word_embeddings.weight)
            for batch in tqdm(validation_loader):
                input_ids1, token_type_ids1, attention_masks1, input_ids2, token_type_ids2, attention_masks2, labels = batch
                input_ids1 = input_ids1.to(device)
                token_type_ids1 = token_type_ids1.to(device)
                attention_masks1 = attention_masks1.to(device)
                input_ids2 = input_ids2.to(device)
                token_type_ids2 = token_type_ids2.to(device)
                attention_masks2 = attention_masks2.to(device)

                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                copy_labels = labels
                with torch.no_grad():
                    preds, male_softmax, female_softmax = network(input_ids1, attention_masks1, token_type_ids1, input_ids2, attention_masks2, token_type_ids2, 'training')
                    labels = labels.view(-1, 1)
                    loss1 = loss_fn(preds, copy_labels)
                    loss2 = loss_fn_2(male_softmax.log(), female_softmax) + loss_fn_2(female_softmax.log(),male_softmax)
                    loss = loss1 + loss2
                    m.track_validation_loss(loss1, loss2, loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
            save_model(network, optimizer)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'Toxicity_KLDM.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return

if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = BATCH_SIZE, fresh_training = True)

,run,epoch,ce_loss,kl_loss,loss,accuracy,validation ce loss,validation kl loss,validation loss,validation accuracy,epoch duration (minutes),run duration (minutes),lr,batch_size,device
0,1,1,0.113629,0.038680,0.152309,0.773593,0.069712,0.005400,0.075113,0.881206,6.125091,6.125146,0.0001,4,cuda
1,1,2,0.064151,0.019109,0.083259,0.891892,0.061311,0.006396,0.067707,0.898936,6.138257,12.369726,0.0001,4,cuda
2,1,3,0.049891,0.014537,0.064428,0.927116,0.056044,0.007574,0.063618,0.907801,6.136359,18.636412,0.0001,4,cuda
3,1,4,0.045651,0.013389,0.059039,0.937971,0.052210,0.008695,0.060904,0.929965,6.137534,24.904331,0.0001,4,cuda
4,1,5,0.041878,0.014118,0.055996,0.944395,0.050249,0.009170,0.059420,0.934397,6.135654,31.170653,0.0001,4,cuda
5,1,6,0.036917,0.012840,0.049757,0.949490,0.047880,0.009189,0.057069,0.934397,6.140576,37.441082,0.0001,4,cuda
6,1,7,0.032308,0.012662,0.044970,0.957244,0.047676,0.009755,0.057432,0.937057,6.149671,43.719971,0.0001,4,cuda
7,1,8,0.030065,0.013274,0.043339,0.957244,0.047761,0.009743,0.057504,0.940603,6.155294,50.009363,0.0001,4,cuda
8,1,9,0.027521,0.011986,0.039507,0.961232,0.048976,0.010123,0.059099,0.941489,6.155293,56.298757,0.0001,4,cuda
9,1,10,0.026024,0.011783,0.037807,0.963004,0.046509,0.010262,0.056771,0.940603,6.156448,62.590088,0.0001,4,cuda


Saved model and optimizer at Toxicity_KLDM.pth
Saved model and optimizer at Toxicity_KLDM.pth


## Load Saved Model

In [1]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if load_and_run:
  load_model(masked_model, None, PATH = '/kaggle/input/toxicity_kldm/pytorch/default/1/Toxicity_KLDM.pth')

## Inference

In [2]:
import random
import warnings
warnings.filterwarnings('ignore')
def get_prediction(data_loader, network, choice = 'original'):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids1, token_type_ids1, attention_masks1, input_ids2, token_type_ids2, attention_masks2, labels = batch
    input_ids1 = input_ids1.to(device)
    token_type_ids1 = token_type_ids1.to(device)
    attention_masks1 = attention_masks1.to(device)
    input_ids2 = input_ids2.to(device)
    token_type_ids2 = token_type_ids2.to(device)
    attention_masks2 = attention_masks2.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        if choice == 'male':
          preds, hidden_states = network(input_ids1, attention_masks1,token_type_ids1, None, None, None, 'inference')
        else:
          preds, hidden_states = network(input_ids2, attention_masks2,token_type_ids2, None, None, None, 'inference')

        labels = labels.type_as(preds).view(-1, 1)
        preds =  preds.max(1)[1].view(-1,1)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()

p, emb_male = get_prediction(test_loader, masked_model, 'male')
test['pred_male_kldm'] = [x[0] for pred in p for x in pred.tolist()]
p, emb_male = get_prediction(test_loader, masked_model, 'female')
test['pred_female_kldm'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['pred_male_kldm']!=test['_original_label'])
print('For Male:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test))) 
x = sum(test['pred_female_kldm']!=test['_original_label'])
print('For Female:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)))
print('Male Female Mismatch: ', sum(test['pred_male_kldm']!=test['pred_female_kldm']))

100%|██████████| 353/353 [00:41<00:00,  8.45it/s]

For Male:
Total test data:  1410 Total mismatch:  131 Accuracy:  0.9070921985815603
For Female:
Total test data:  1410 Total mismatch:  146 Accuracy:  0.8964539007092198
Male Female Mismatch:  43


# KLD Minimization Result

In [4]:
import numpy as np
np.random.seed(0)

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []

    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])

    return original_acc, (lower, upper), np.mean(bootstrap_accs)

y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['pred_male_kldm'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

y_true = test['_original_label'].values
y_pred = test['pred_female_kldm'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")
print('Male Female Mismatch: ', sum(test['pred_male_kldm']!=test['pred_female_kldm']))
test.to_csv('revision_kldm_toxicity_result.csv', index = False)
print('Saved....')

Male: 
Accuracy: 0.9071
95% Confidence Interval: (0.8918262411347517, 0.9223581560283688)
mean accuracy: 0.9068921985815602

Female: 
Accuracy: 0.8965
95% Confidence Interval: (0.8818971631205674, 0.9113475177304965)
mean accuracy: 0.896463829787234
Male Female Mismatch:  43
Saved....


In [1]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('revision_kldm_toxicity_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.pred_male_kldm.values
female_pred = test.pred_female_kldm.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

c:\Users\joyau\anaconda3\envs\pytorch_gpu\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


SPD 95% Confidence Interval: [0.000, 0.011]	SPD Mean: 0.004
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.001, 0.012]	EOD Mean: 0.006


## Result of Approach 2

In [ ]:
x = sum(test['original_lebel'] != test['a2_original_pred'])
print('For Original text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )
x = sum(test['original_lebel'] != test['a2_ner_pred'])
print('For NER Converted text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)))
print('Bias: ', sum(test['a2_ner_pred'] != test['a2_original_pred']))

In [ ]:
x = sum(test['original_lebel'] != test['a2_original_pred'])
print('For Original text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )
x = sum(test['original_lebel'] != test['a2_ner_pred'])
print('For NER Converted text:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)))
print('Bias: ', sum(test['a2_ner_pred'] != test['a2_original_pred']))

In [ ]:
x = test[test['a2_ner_pred'] != test['a2_original_pred']][['Original Sentence', 'Converted Sentence', 'original_lebel', 'a2_original_pred', 'a2_ner_pred']]
print(len(x))
x

## Sentence Embedding Visualization for Original Sentence and NER Converted Sentence

### For Training Data

In [ ]:
#for visualizing training data embedding
dataset_train= BertDataset(train, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle = False)
p, emb_ori = get_prediction(train_loader, masked_model, 'original')
p, emb_ner = get_prediction(train_loader, masked_model, 'ner')

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ori)

labels = train['original_lebel']   # Extract labels
review_texts = train['Original Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings Original Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ner)

labels = train['original_lebel']   # Extract labels
review_texts = train['Converted Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings NER Converted Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()


### For Biased Test Data

In [ ]:
#for visualizing training data embedding
dataset_train= BertDataset(x, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle = False)
p, emb_ori = get_prediction(train_loader, masked_model, 'original')
p, emb_ner = get_prediction(train_loader, masked_model, 'ner')

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ori)

labels = x['original_lebel']   # Extract labels
review_texts = x['Original Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings Original Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_cls = TSNE(n_components=2, random_state=42).fit_transform(emb_ner)

labels = x['original_lebel']   # Extract labels
review_texts = x['Converted Sentence']  # Extract review texts

# Prepare the data for Plotly plots
plot_data_cls = {'t-SNE Dimension 1': tsne_cls[:, 0],
                 't-SNE Dimension 2': tsne_cls[:, 1],
                 'Label': labels,
                 'Text': review_texts}

# Customize colorbar to show integer labels
colorbar_tickvals = list(range(1, 6))
colorbar_ticktext = [str(val) for val in colorbar_tickvals]

# Plotting with Plotly - [CLS] Token Embeddings
fig_cls = px.scatter(plot_data_cls, x='t-SNE Dimension 1', y='t-SNE Dimension 2',
                     color='Label', hover_data={'t-SNE Dimension 1': False, 't-SNE Dimension 2': False, 'Text': True})


fig_cls.update_layout(title='t-SNE of Embeddings NER Converted Training Data with Labels', height=700,  font=dict(
        size=20,
))
fig_cls.update_traces(marker=dict(size=10, opacity=0.9))

fig_cls.update_layout(coloraxis_colorbar=dict(
    tickvals=colorbar_tickvals,
    ticktext=colorbar_ticktext
))

fig_cls.show()

In [ ]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/toxicity_reSult_a2.csv', index = False)